In [20]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import datetime
import time
import re
import pytz

import django
django.db.close_old_connections()



query = IntimacaoRecebida.objects.all().prefetch_related('informacao',
                                                        'parte_citada',
                                                        'parte_oposta',
                                                        'intimacao',
                                                        'unidade',
                                                        'usuario')

context = []

for m in query:
    entrada = {
               '<td>{entrada.ordem}</td>': m.id,
               '<td>{entrada.processo}</td>': m.informacao.processo,
               '<td>{entrada.parte_citada}</td>': m.parte_citada.nome,
               '<td>{entrada.parte_oposta}</td>': m.parte_oposta.nome,
               '<td>{entrada.intimacao}</td>': m.intimacao.codigo_validacao,
               '<td>{entrada.dia}</td>': datetime.datetime.strftime(m.hora_citacao, '%H:%M:%S'),
               '<td>{entrada.hora}</td>': datetime.datetime.strftime(m.hora_citacao, '%d/%m/%Y'),
               '<td>{entrada.unidade}</td>': m.unidade.unidade,
               '<td>{entrada.SEI}</td>': m.sei_gerado,
               '<td>{entrada.usuário}</td>': m.usuario.username,
               }
    context += [entrada]
    
context = {'data': context}



{'data': [{'ordem': 1,
   'processo': '1063156-53.2018.8.26.0053',
   'parte_citada': 'DEPARTAMENTO ESTADUAL DE TRÂNSITO DE SÃO PAULO - DETRAN',
   'parte_oposta': 'DEPARTAMENTO ESTADUAL DE TRÂNSITO DE SÃO PAULO - DETRAN',
   'intimacao': '05320190137623',
   'dia': '01:38:41',
   'hora': '17/09/2019',
   'unidade': 'JUD 5',
   'SEI': None,
   'usuário': 'ezequiel'},
  {'ordem': 2,
   'processo': '1063156-53.2018.8.26.0053',
   'parte_citada': 'DEPARTAMENTO ESTADUAL DE TRÂNSITO DE SÃO PAULO - DETRAN',
   'parte_oposta': 'DEPARTAMENTO ESTADUAL DE TRÂNSITO DE SÃO PAULO - DETRAN',
   'intimacao': '05320190137623',
   'dia': '01:40:44',
   'hora': '17/09/2019',
   'unidade': 'JUD 5',
   'SEI': None,
   'usuário': 'ezequiel'},
  {'ordem': 3,
   'processo': '1063535-91.2018.8.26.0053',
   'parte_citada': 'Prefeitura do Municipio de São Paulo',
   'parte_oposta': 'Prefeitura do Municipio de São Paulo',
   'intimacao': '05320190653567',
   'dia': '21:13:31',
   'hora': '17/09/2019',
   'unidad

In [93]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import datetime
import time
import re
import pytz
import requests
import tempfile

s = requests.session()

if IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).exists():

    m = IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).prefetch_related('informacao',
                                                                                   'parte_citada',
                                                                                   'parte_oposta',
                                                                                   'intimacao',
                                                                                   'unidade'
                                                                                  ).first()
    documento = Documentos.objects.get(intimacao=m.intimacao)
    r = s.get(documento.link_doc, verify=False)

    temp_int = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False)
    with temp_int as file:
        file.write(r.content)
    documento = temp_int.name

    #interessado = m.parte_oposta.nome
    #cnj = m.informacao.processo


    driver = webdriver.Chrome(r'/home/ezequiel/Desktop/chromedriver')

    # tela de login
    driver.get('https://sip.prefeitura.sp.gov.br/sip/login.php?sigla_orgao_sistema=PMSP&sigla_sistema=SEI')
    driver.find_element_by_id('txtUsuario').send_keys('d858414')
    time.sleep(0.1)
    driver.find_element_by_id('pwdSenha').send_keys('013451Ab')
    select  = Select(driver.find_element_by_id('selOrgao'))
    select.select_by_value('11')
    driver.find_element_by_id('sbmLogin').click()

    # iniciar processo
    driver.find_element_by_xpath('//*[@id="main-menu"]/li[3]/a').click()
    # Comunicação de decisão judicial e pedido de informação
    driver.find_element_by_xpath('//*[@tabindex="1003"]').click()
    # Parametros da tela
    driver.find_element_by_id('txtDescricao').send_keys(m.informacao.processo)

    # Assunto
    driver.find_element_by_id('imgPesquisarAssuntos').click()
    driver.switch_to_window(driver.window_handles[1])
    driver.find_element_by_id('txtPalavrasPesquisaAssuntos').send_keys('adm civ')
    driver.find_element_by_id('btnPesquisar').click()
    driver.find_element_by_xpath('//*[@id="lnkInfraT-1325"]/img').click()
    driver.switch_to_window(driver.window_handles[0])

    # Interessado

    driver.find_element_by_id('txtInteressadoProcedimento').send_keys(m.parte_oposta.nome)
    driver.find_element_by_id('txtInteressadoProcedimento').send_keys(Keys.ENTER)
    try:
        alert = driver.switch_to.alert
        alert.accept()
    except:
        pass

    # doc restrito
    driver.find_element_by_id('optRestrito').click()
    while 'Documento Preparatório' not in driver.page_source:
        time.sleep(0.1)
    driver.find_element_by_id('btnSalvar').click()

    # pagina seguinte
    driver.switch_to_frame(driver.find_element_by_id('ifrArvore'))
    id_procedimento = re.search('(?<=id_procedimento=)\d+', driver.current_url).group()
    m.sei_gerado = driver.find_element_by_id(f'span{id_procedimento}').text

    # inserir documento
    driver.switch_to_default_content()
    driver.switch_to_frame(driver.find_element_by_id('ifrVisualizacao'))
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="divArvoreAcoes"]/a[1]/img').click()
    driver.find_element_by_xpath('//*[@id="tblSeries"]/tbody/tr[1]/td/a[2]').click()


    # # mandado de citação
    select = Select(driver.find_element_by_id('selSerie'))
    select.select_by_value('2188')
    # # data do mandado de citação
    driver.find_element_by_id('txtDataElaboracao').send_keys(datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Brazil/East')), '%d/%m/%Y'))
    # # nato-digital
    driver.find_element_by_id('lblNato').click()
    # # restrito
    driver.find_element_by_id('lblRestrito').click()

    driver.find_element_by_id('filArquivo').send_keys(documento)

    driver.find_element_by_id('btnSalvar').click()
    
    m.save()


#     driver.find_element_by_xpath('//*[@id="span22966958"]').click()


/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: use driver.switch_to.window instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:57: DeprecationWarning: use driver.switch_to.window instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: use driver.switch_to.frame instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:81: DeprecationWarning: use driver.switch_to.default_content instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: use driver.switch_to.frame instead


In [88]:
Informacoes.objects.all().delete()

(101,
 {'core.Informacoes_unidade': 0,
  'core.Informacoes_objeto': 0,
  'core.Informacoes_complementoobjeto': 0,
  'core.Movimentacoes': 0,
  'core.IntimacaoRecebida': 0,
  'core.Documentos': 100,
  'core.Informacoes': 1})

In [87]:
b'\xc3\xa7'.decode('utf8').encode(encoding='latin1').decode('latin1')

'ç'

In [64]:
import django
django.db.close_old_connections()
from core.models import *
import tempfile
import requests


from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import datetime
import time
import re
import pytz


s = requests.session()

if IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).exists():
    
    m = IntimacaoRecebida.objects.filter(sei_gerado__isnull=True).prefetch_related('informacao',
                                                                                   'parte_citada', 
                                                                                   'parte_oposta',
                                                                                   'intimacao',
                                                                                   'unidade'
                                                                                  ).first()
    documento = Documentos.objects.get(intimacao=m.intimacao)
    r = s.get(documento.link_doc, verify=False)
    
    temp_int = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False)
    with temp_int as file:
        file.write(r.content)
    documento = temp_int.name
    
    #interessado = m.parte_oposta.nome
    #cnj = m.informacao.processo
    

    driver = webdriver.Chrome(r'/home/ezequiel/Desktop/chromedriver')
    
    # tela de login
    driver.get('https://sip.prefeitura.sp.gov.br/sip/login.php?sigla_orgao_sistema=PMSP&sigla_sistema=SEI')
    driver.find_element_by_id('txtUsuario').send_keys('d858414')
    time.sleep(0.1)
    driver.find_element_by_id('pwdSenha').send_keys('013451Ab')
    select  = Select(driver.find_element_by_id('selOrgao'))
    select.select_by_value('11')
    driver.find_element_by_id('sbmLogin').click()
    
    # iniciar processo
    driver.find_element_by_xpath('//*[@id="main-menu"]/li[3]/a').click()
    # Comunicação de decisão judicial e pedido de informação
    driver.find_element_by_xpath('//*[@tabindex="1003"]').click()
    # Parametros da tela
    driver.find_element_by_id('txtDescricao').send_keys(m.informacao.processo)
    
    # Assunto
    driver.find_element_by_id('imgPesquisarAssuntos').click()
    driver.switch_to_window(driver.window_handles[1])
    driver.find_element_by_id('txtPalavrasPesquisaAssuntos').send_keys('adm civ')
    driver.find_element_by_id('btnPesquisar').click()
    driver.find_element_by_xpath('//*[@id="lnkInfraT-1325"]/img').click()
    driver.switch_to_window(driver.window_handles[0])

    # Interessado

    driver.find_element_by_id('txtInteressadoProcedimento').send_keys(m.parte_oposta.nome)
    driver.find_element_by_id('txtInteressadoProcedimento').send_keys(Keys.ENTER)
    try:
        alert = driver.switch_to.alert
        alert.accept()
    except:
        pass
    
    # doc restrito
    driver.find_element_by_id('optRestrito').click()
    while 'Documento Preparatório' not in driver.page_source:
        time.sleep(0.1)
    driver.find_element_by_id('btnSalvar').click()
    
    # pagina seguinte
    driver.switch_to_frame(driver.find_element_by_id('ifrArvore'))
    id_procedimento = re.search('(?<=id_procedimento=)\d+', driver.current_url).group()
    m.sei_gerado = driver.find_element_by_id(f'span{id_procedimento}').text
    
    # inserir documento
    driver.switch_to_default_content()
    driver.switch_to_frame(driver.find_element_by_id('ifrVisualizacao'))
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="divArvoreAcoes"]/a[1]/img').click()
    driver.find_element_by_xpath('//*[@id="tblSeries"]/tbody/tr[1]/td/a[2]').click()


    # # mandado de citação
    select = Select(driver.find_element_by_id('selSerie'))
    select.select_by_value('2188')
    # # data do mandado de citação
    driver.find_element_by_id('txtDataElaboracao').send_keys(datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Brazil/East')), '%d/%m/%Y'))
    # # nato-digital
    driver.find_element_by_id('lblNato').click()
    # # restrito
    driver.find_element_by_id('lblRestrito').click()

    driver.find_element_by_id('filArquivo').send_keys(documento)
    
    driver.find_element_by_id('btnSalvar').click()
    
    
    driver.find_element_by_xpath('//*[@id="span22966958"]').click()

/home/ezequiel/pgm/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:59: DeprecationWarning: use driver.switch_to.window instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:63: DeprecationWarning: use driver.switch_to.window instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: use driver.switch_to.frame instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:87: DeprecationWarning: use driver.switch_to.default_content instead
/home/ezequiel/pgm/venv/lib/python3.7/site-packages/ipykernel_launcher.py:88: DeprecationWarning: use driver.switch_to.frame instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="span22966958"]"}
  (Session info: chrome=77.0.3865.75)


In [65]:
driver.find_element_by_xpath('//*[@id="span22966958"]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="span22966958"]"}
  (Session info: chrome=77.0.3865.75)


In [18]:



# # mandado de citação
select = Select(driver.find_element_by_id('selSerie'))
select.select_by_value('2188')
# # data do mandado de citação
driver.find_element_by_id('txtDataElaboracao').send_keys(datetime.datetime.strftime(datetime.datetime.now(), '%d/%m/%Y'))
# # nato-digital
driver.find_element_by_id('lblNato').click()
# # restrito
driver.find_element_by_id('lblRestrito').click()



# driver.find_element_by_id('btnSalvar').click()

# # driver.find_element_by_id(f'span{id_procedimento}').click()

    

In [14]:
params = {'origem': 'criar_processo', 'intimacao': '2', 'parte_contraria': '4', 'parte_citada': '4', 'unidade': '1', 'csrfmiddlewaretoken': 'PXmmfXAfn5cEQK0BHw0b0Yk1OwrmrmynPFEW32p8MP573f2F1HZ8j3w77fZ9i6gV'}



m = IntimacaoRecebida()
m.parte_citada = Partes.objects.get(pk=params['parte_citada'])
m.parte_oposta = Partes.objects.get(pk=params['parte_contraria'])
intimacao = Intimacao.objects.get(pk=params['intimacao'])
m.intimacao = intimacao
m.informacao = Informacoes.objects.get(pk=intimacao.informacao_id)
m.unidade = Unidade.objects.get(pk=params['unidade'])
m.save()

In [9]:
intimacao = Intimacao.objects.get(pk=params['intimacao'])
Informacoes.objects.get(pk=intimacao.informacao_id)


<Informacoes: 10631565320188260053>

In [27]:
    informacao = models.ForeignKey(Informacoes, on_delete=models.CASCADE)
    parte_citada = models.ForeignKey(Partes, on_delete=models.CASCADE, related_name='p_citada')
    parte_oposta = models.ForeignKey(Partes, on_delete=models.CASCADE, related_name='p_oposta')
    intimacao = models.ForeignKey(Intimacao, on_delete=models.CASCADE)
    hora_citacao = models.DateTimeField(auto_now_add=True)
    unidade = models.ForeignKey(Unidade, on_delete=models.CASCADE)
    sei_gerado = models.CharField(max_length=255, null=True)

    urgente = models.BooleanField(default=False)

In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import datetime
import time
import re

# driver = webdriver.Chrome(r'C:\Anaconda3\chromedriver_win32\chromedriver.exe')
driver = webdriver.Chrome(r'/home/ezequiel/Desktop/chromedriver')

cnj = 123456
interessado = 'Interessados3'

# tela de login
driver.get('https://sip.prefeitura.sp.gov.br/sip/login.php?sigla_orgao_sistema=PMSP&sigla_sistema=SEI')
driver.find_element_by_id('txtUsuario').send_keys('d858414')
time.sleep(0.1)
driver.find_element_by_id('pwdSenha').send_keys('013450Ab')
select  = Select(driver.find_element_by_id('selOrgao'))
select.select_by_value('11')
driver.find_element_by_id('sbmLogin').click()

# iniciar processo
driver.find_element_by_xpath('//*[@id="main-menu"]/li[3]/a').click()

# Comunicação de decisão judicial e pedido de informação
driver.find_element_by_xpath('//*[@id="tblTipoProcedimento"]/tbody/tr[9]/td/a[2]').click()

# Parametros da tela
driver.find_element_by_id('txtDescricao').send_keys(cnj)

# Assunto
driver.find_element_by_id('imgPesquisarAssuntos').click()
driver.switch_to_window(driver.window_handles[1])
driver.find_element_by_id('txtPalavrasPesquisaAssuntos').send_keys('adm civ')
driver.find_element_by_id('btnPesquisar').click()
driver.find_element_by_xpath('//*[@id="lnkInfraT-1325"]/img').click()
driver.switch_to_window(driver.window_handles[0])

time.sleep(0.1)

# Interessado

driver.find_element_by_id('txtInteressadoProcedimento').send_keys(interessado)
driver.find_element_by_id('txtInteressadoProcedimento').send_keys(Keys.ENTER)
try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    pass

driver.find_element_by_id('optRestrito').click()
while 'Documento Preparatório' not in driver.page_source:
    time.sleep(0.1)
driver.find_element_by_id('btnSalvar').click()

# nova pagina
a = driver.page_source
driver.switch_to_frame(driver.find_element_by_id('ifrArvore'))
id_procedimento = re.search('(?<=id_procedimento=)\d+', driver.current_url).group()
numero_sei = driver.find_element_by_id(f'span{id_procedimento}').text

# inserir documento
driver.switch_to_default_content()
driver.switch_to_frame(driver.find_element_by_id('ifrVisualizacao'))
time.sleep(1)
driver.find_element_by_xpath('//*[@id="divArvoreAcoes"]/a[1]/img').click()
driver.find_element_by_xpath('//*[@id="tblSeries"]/tbody/tr[1]/td/a[2]').click()

# # mandado de citação
select = Select(driver.find_element_by_id('selSerie'))
select.select_by_value('2188')
# # data do mandado de citação
driver.find_element_by_id('txtDataElaboracao').send_keys(datetime.datetime.strftime(datetime.datetime.now(), '%d/%m/%Y'))
# # nato-digital
driver.find_element_by_id('lblNato').click()
# # restrito
driver.find_element_by_id('lblRestrito').click()

driver.find_element_by_id('filArquivo').send_keys('/home/ezequiel/Desktop/sample.pdf')

driver.find_element_by_id('btnSalvar').click()

driver.find_element_by_id(f'span{id_procedimento}').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-menu"]/li[3]/a"}
  (Session info: chrome=77.0.3865.75)


In [37]:
from utils import login, scrapper
import re

s = login.get_esaj_logged_session()

x = '1041144-11.2019.8.26.0053'
x = re.sub('\D','',x)
    
res = {'nuprocessounificadoformatado': x,
       'fonte_comunicacao' : 'usuario',
       'metodo_comunicacao': 'intimacao',
       'razao_inclusao': 'indefinido'}

res['versao_parser'] = 5.1

params = {'cbPesquisa': 'NUMPROC',
          'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
          'dadosConsulta.valorConsultaNuUnificado': res['nuprocessounificadoformatado']}

r = s.get('https://esaj.tjsp.jus.br/cpopg/search.do', params=params, verify=False)
sp = bs(r.content, 'html5lib')

res['versao_parser'] = 5.1

if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:

    res['digital'] = True

    crit = re.compile(r'requestScope = (.*?);\s+?var', re.I | re.DOTALL)
    parametro = re.search(r'.codigo=(.*?)&', r.url).group(1)

    s.get('https://esaj.tjsp.jus.br/cpopg/show.do?processo.foro=53&processo.codigo=%s&gateway=true' % parametro)

    r = s.get('https://esaj.tjsp.jus.br/cpopg/abrirPastaDigital.do?processo.codigo=%s' % parametro, verify=False)

    j = json.loads(re.search(crit, r.text).group(1))

    coleta = []

    c_assinatura = re.compile(
        'p\s{,4}a\s{,4}r\s{,4}a\s{,4}c\s{,4}o\s{,4}n\s{,4}f\s{,4}e\s{,4}r\s{,4}i\s{,4}r\s{,4}.*?fls.{,4}\d+',
        re.I | re.DOTALL)

    for linha in j:
        for sublinha in linha['children']:
            print('AAAAAAA')
            rp = {
                'indicepagina': sublinha['data']['indicePagina'],
                'title': linha['data']['title'],
                'cddocumento': linha['data']['cdDocumento'],
                'dtinclusao': datetime.datetime.strptime(linha['data']['dtInclusao'], '%d/%m/%Y %H:%M:%S'),
                'cdtipodocdigital': linha['data']['cdTipoDocDigital'],
                'parametros': sublinha['data']['parametros'].replace('amp;', ''),
            }

            texto_doc = s.get('https://esaj.tjsp.jus.br/pastadigital/getPDF.do?' + rp['parametros'], verify=False)
            texto_doc = parser.from_buffer(io.BytesIO(texto_doc.content), settings.TIKA_SERVER)['content']

            if texto_doc:
                texto_doc = re.sub(c_assinatura, '', texto_doc).strip()

                if len(texto_doc) > 100 and texto_doc[:7] != 'Atenção':
                    rp['conteudo'] = texto_doc.encode(encoding='latin1', errors='ignore').decode(encoding='latin1')

            
            coleta += [rp]

    if len(coleta) > 0:
        res['documentos'] = coleta

if "mostrar: 'true'" in str(sp):
    res['segredo_de_justica'] = True

else:

    # diversos

    for item in sp.findAll(
            attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
        if item.get('for') == 'tag.dados.rotulo.principal':
            res['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

        elif item.get_text() == 'Distribuição:':
            res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            rp = item.findNext('span').findNext()
            if rp.find('span') is not None:
                res['vara'] = rp.find('span').get_text().strip()

        elif item.get_text() == 'Apensado ao:':
            res[item.get_text().strip().replace(':', '')] = item.findNext(
                attrs={'class': 'processoPaiApenso'}).get_text().strip()

        elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
            res['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
        else:
            res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

        if item.get_text() == 'Recebido em:':
            res['vara'] = item.findNext('span').findNext('span').get_text()

    for i in sp.findAll('span', attrs={'class': 'labelClass'}):
        res[i.get_text().replace(':', '')] = i.next.next.strip()

    # extinto_suspenso
    if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
        res['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

    if 'Distribuição' in res.keys():
        rp = res['Distribuição'][:10]
        res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()
    elif 'Recebido em' in res.keys():
        rp = res['Recebido em'][:10]
        res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()

    if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
        res['tramitacao_prioritaria'] = 'Tramitação prioritária'
    if 'Valor da ação' in res.keys():
        res['Valor da ação'] = re.sub(r'\s\s+', ' ', res['Valor da ação'])
        if re.match(r'(.*?)\s', res['Valor da ação']) is not None:
            res['moeda'] = re.match(r'(.*?)\s', res['Valor da ação']).group(1)
        if re.search(r'\s(.*)', res['Valor da ação']) is not None:
            rp = re.search(r'\s(.*)', res['Valor da ação']).group(1).strip()
            res['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

        del res['Valor da ação']

    # partes
    coleta = []
    controle = []

    for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top',
                                        'style': 'padding-bottom: 5px'}):
        rp = {}

        if item.findNext('td').next.strip() not in controle:
            controle += [item.findNext('td').next.strip()]

            rp['nome'] = item.findNext('td').next.strip()

            rp['posicao'] = item.get_text().strip().replace(':', '')

            procuradores = []

            for subitem in item.findNextSibling().findAll('span'):
                classe = subitem.get_text().replace(':', '').strip()
                nome = subitem.next.next.strip()
                procuradores += [[classe, nome]]

            classes = list(set([i[0] for i in procuradores]))
            subcoleta = {}

            for classe in classes:
                nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                subcoleta[classe] = nome_procurador

            if len(subcoleta) > 0:
                rp['procuradores'] = subcoleta
            coleta += [rp]

    if len(coleta) > 0:
        res['partes'] = coleta

    # movimentacoes
    coleta = []
    lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
    r = list(range(len(lista) - 1, -1, -1))

    for n, item in zip(r, lista):
        entrada = {'ordem': n,
                   'data': datetime.datetime.strptime(item.get_text().strip(), '%d/%m/%Y').date()}

        if len(item.findNext('td').findChildren()) > 0:
            entrada['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
            entrada['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
        else:

            entrada['tag'] = item.findNext('td').findNext('td').next.strip()
            entrada['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
        coleta += [entrada]

    if len(coleta) > 0:
        res['movimentacoes'] = coleta

res = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in res.items() if v}
docinformacao = Informacoes(**{k: v for k, v in res.items() if k not in ['movimentacoes', 'partes', 'documentos']})
docinformacao.save()
if 'documentos' in res.keys():
    coleta = []
    for entrada_doc in res['documentos']:
        coleta += [Documentos(**entrada_doc, informacao=docinformacao)]
    Documentos.objects.bulk_create(coleta)

if 'partes' in res.keys():
    coleta = []
    for entrada_doc in res['partes']:
        coleta += [Partes(**entrada_doc, informacao=docinformacao)]
    Partes.objects.bulk_create(coleta)

if 'movimentacoes' in res.keys():
    coleta = []
    for entrada_doc in res['movimentacoes']:
        coleta += [Movimentacoes(**entrada_doc, informacao=docinformacao)]
    Movimentacoes.objects.bulk_create(coleta)

documentosquery = Documentos.objects.filter(informacao=docinformacao, title='Mandados')
crit = re.compile(r'\*(\d{,14})\*', re.DOTALL)
for documento in documentosquery:
    if isinstance(documento.conteudo, str):
        if re.search(crit, documento.conteudo):
            cod = re.search(crit, documento.conteudo).group(1)
            intimacao = Intimacao(codigo_validacao=cod, documento=documento, informacao=docinformacao)
            intimacao.save()



AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA
AAAAAAA


<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>

	








	<script language="javascript" src="https://esaj.tjsp.jus.br/sajcas/verificarLogin.js?script=1568658741560" type="text/JavaScript"></script>

	<script language="javascript">
		if(window.sajcas && window.sajcas.usuarioLogadoNoCasServer) {
			var urlRetornoSistema = '';
			if(urlRetornoSistema === '') {
				urlRetornoSistema = window.location.href;
			}

			if(urlRetornoSistema.lastIndexOf('?') != -1) {
				urlRetornoSistema += '&';
			} else {
				urlRetornoSistema += '?';
			}
			urlRetornoSistema+= 'gateway=true';
			window.location.href = urlRetornoSistema;
		}
	</script>


	<title>Portal de Serviços e-SAJ</title>
	































<!-- Includes do spw -->

<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,func

In [36]:
r = s.get('https://esaj.tjsp.jus.br/cpopg/search.do', params=params, verify=False)
r.url

'https://esaj.tjsp.jus.br/cpopg/show.do?processo.codigo=1H0006OY30000&processo.foro=53&uuidCaptcha=sajcaptcha_8ef93e9641ae4ad3a61fbaad099ca4f6'